# 기본과제 요구사항 
이전 이미지 관련 실습에서는 사용자가 이미지를 업로드하면 하드 코딩 된 prompt와 함께 GPT로 보내 답변을 받는 챗봇을 구현했습니다. 이번에는 다음과 같이 기능을 확장하는 것이 과제의 목표입니다:

- 여러 이미지를 입력으로 받기
    - 기존에는 이미지 한 장만을 입력으로 받았다면 이번에는 다수의 사진을 입력 받을 수 있도록 만들어야 합니다.
    - Streamlit의 `st.file_uploader` [문서](https://docs.streamlit.io/develop/api-reference/widgets/st.file_uploader)를 확인하여 기존 코드에서 여러 장의 사진을 받을 수 있도록 구현해봅시다.
- 업로드 된 이미지들을 가지고 자유롭게 질의응답 할 수 있는 챗봇 구현
    - 기존과 다르게 하드코딩 된 prompt가 아닌, 사용자로부터 질문을 입력받아 GPT에게 넘겨주는 챗봇을 구현하셔야 합니다.
    - 그리고 사용자가 여러 번 질문을 입력해도 처음 주어진 이미지들로 답변할 수 있도록 구현하셔야 합니다(이 부분은 RAG 실습 코드를 참조).
    - GPT에게 여러 개의 사진을 보내주는 부분은 [API 문서](https://platform.openai.com/docs/guides/vision)를 확인하여 구현하시면 됩니다.
- 다음 이미지들과 질문에 대한 챗봇의 답변 생성
    - 다음 주어진 이미지들과 질문을 실제로 구현한 챗봇에게 주어졌을 때 어떤 답변이 생성되는지 영상을 녹화하셔야 합니다:
        - 이미지: 인터넷에서 강아지 사진과 고양이 사진 각각 1장씩 찾아 입력으로 쓰시면 됩니다.
        - 질문 1: 주어진 두 사진의 공통점이 뭐야?
        - 질문 2: 주어진 두 사진의 차이점이 뭐야?
    - 질문 1, 2를 차례로 입력하시면 됩니다. 즉, 챗봇과 질의응답이 두 차례 이루어져야 합니다.


#

In [1]:
import streamlit as st
import base64
from PIL import Image
from io import BytesIO
import numpy as np
import faiss
import torch
from transformers import CLIPProcessor, CLIPModel
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv
import os

# OpenAI API 환경 변수 로드

In [4]:
from dotenv import load_dotenv

load_dotenv()

True

[MYCODE]
- 이미지 업로드시 base64로 변경하여 이미지를 업로드합니다. 
- 동일한 이미지인지 판단 FAISS vectorstore를 통해 판단하였습니다.
- CLIP 모델을 사용하여 이미지를 벡터화 하였습니다.
- FAISS 를 이용해서 이미지를 벡터 DB 에 저장하여 이미지 동일여부를 판단하였습니다. 

여러 고민을 해보고 있기는 하지만, 맞는지를 모르겠습니다. 
이런 방법이 가능하다고 생각을 하면 OPENAI의 응답 정보를 정규화해서 회사내 모델을 만들 수 있겠다는 생각이들어 그냥 생각이 나는대로 질문을 던져 봅니다. 
조금은 이해부탁드려요 

[FEEDBACK] 
- 과제 제출 시점에 드는 생각은 이미지의 정보를 벡터 디비에 저장하는 것보다 오픈 AI가 대답해주는 이미지의 공통점을 벡터스토어 저장해두고서    
유사한 질문은 응답 해주면 좋겠다는 생각이 들다가 같은 강아지여도 사진이 다르다면 안되겠구나 생각에 **업로드 이미지** 를 FAISS 에 저장 OPENAI로 견종의 특징을 ChromaDB 에 저장하는 생각도 해보게 되었습니다.

위 생각을 통해 아래와 같은 질문이 생겼습니다.
1. 응답정보를 벡터디비에 저장해두고서 유사한 질문은 벡터디비의 정보로 응답해주는 방법을 많이 사용하나요? 
- 이미지의 경우는 쉽지 않아 보이지만, 문자 같은 경우 가능할 것 같다는 생각이 들었습니다. 

2.  **업로드 이미지** 를 FAISS 에 저장 OPENAI로 견종의 특징을 ChromaDB 에 저장
- 위 방식에서 FAISS에 이미지를 업로드 하더라도 해당 이미지를 찾는 역할 밖에 못할거 같은데 ChromaDB에 저장시 FAISS 이미지 key 값을 함께 저장하면 되는 걸까요?

3. 이런 고민을 하는 커뮤니티를 알면 좋을거 같기도 해서 작성해보았습니다. 허깅페이스가 대표적인 커뮤니티인거 같기도한데 더 추천해주실만한 곳이 있을까요?

In [ ]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# OpenAI 모델 초기화
llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=OPENAI_API_KEY)

# GPU 지원 여부 확인
device = "cuda" if torch.backends.cuda.is_available() else "cpu"

# CLIP 모델 로드 (세션 유지하여 메모리 절약)
if "clip_model" not in st.session_state:
    st.session_state.clip_model = CLIPModel.from_pretrained(
        "openai/clip-vit-base-patch32"
    ).to(device)
    st.session_state.clip_processor = CLIPProcessor.from_pretrained(
        "openai/clip-vit-base-patch32"
    )

# FAISS 벡터 저장소 초기화 (한 번만 실행)
if "vector_db" not in st.session_state:
    faiss_index = faiss.IndexFlatL2(512)  # 512차원 벡터 저장
    st.session_state.vector_db = faiss_index
    st.session_state.image_metadata = {}  # FAISS에 저장된 이미지 정보 (파일명 저장)

# 채팅 메시지 저장소 초기화 (채팅 기록 유지)
if "messages" not in st.session_state:
    st.session_state.messages = []


clip_model = st.session_state.clip_model
clip_processor = st.session_state.clip_processor


# CLIP을 사용하여 이미지 벡터화
def get_image_embedding(image: Image.Image) -> np.ndarray:
    inputs = clip_processor(images=image, return_tensors="pt").to(device)
    with torch.no_grad():
        image_features = clip_model.get_image_features(**inputs)
    return image_features.cpu().numpy().flatten()


# 이미지를 Base64 인코딩하는 함수
def encode_image_to_base64(image_data):
    return base64.b64encode(image_data).decode("utf-8")


# FAISS에서 중복 이미지 확인
def find_duplicate_image(image_vector):
    """FAISS에서 중복된 이미지가 있는지 확인하고, 있으면 해당 인덱스를 반환"""
    if st.session_state.vector_db.ntotal == 0:
        return None  # 저장된 벡터가 없으면 중복 아님

    image_vector = np.array(image_vector, dtype=np.float32).reshape(1, -1)
    D, I = st.session_state.vector_db.search(image_vector, 1)  # 가장 가까운 벡터 검색

    if D[0][0] < 0.001:  # 거리가 0에 가까우면 동일한 이미지로 판단
        return I[0][0]  # 중복된 이미지의 인덱스 반환
    return None


# 이미지 업로드
uploaded_files = st.file_uploader(
    "이미지를 업로드 해주세요 (여러 개 가능)",
    type=["jpg", "jpeg", "png"],
    accept_multiple_files=True,
)

if uploaded_files:
    for file in uploaded_files:
        image = Image.open(file)
        buffer = BytesIO()
        image.save(buffer, format="PNG")

        # CLIP 벡터 생성
        image_vector = get_image_embedding(image)
        image_vector = np.array(image_vector, dtype=np.float32).reshape(1, -1)

        # 중복된 이미지인지 확인
        duplicate_index = find_duplicate_image(image_vector)
        if duplicate_index is not None:
            st.info(
                f"{file.name} 이미지는 이미 업로드된 이미지와 동일합니다. 기존 이미지를 유지합니다."
            )
        else:
            # FAISS에 벡터 추가
            st.session_state.vector_db.add(image_vector)

            # 이미지 정보를 저장 (Base64 인코딩 포함)
            image_base64 = encode_image_to_base64(buffer.getvalue())
            st.session_state.image_metadata[st.session_state.vector_db.ntotal - 1] = {
                "name": file.name,
                "base64": image_base64,
            }

        st.image(image, caption=f"업로드된 이미지: {file.name}", use_column_width=True)

    st.success(
        f"{len(uploaded_files)}개의 이미지가 업로드되었습니다. 질문을 입력하세요!"
    )

# 기존 대화 기록 표시 (채팅 기록 유지)
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# 사용자 질문 처리
prompt = st.chat_input("이미지에 대해 질문을 입력하세요 (예: 공통점은 무엇인가요?)")

if prompt:
    # 사용자 입력 저장 및 표시
    with st.chat_message("user"):
        st.markdown(prompt)
    st.session_state.messages.append({"role": "user", "content": prompt})

    # 업로드된 모든 이미지 정보를 OpenAI에게 전달 (Base64 포함)
    if len(st.session_state.image_metadata) > 0:
        image_info = []
        for i, idx in enumerate(st.session_state.image_metadata):
            image_data = st.session_state.image_metadata[idx]
            image_info.append(
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{image_data['base64']}"
                    },
                }
            )
        image_text = "\n".join(
            [
                f"이미지 {i+1}: {st.session_state.image_metadata[idx]['name']}"
                for i, idx in enumerate(st.session_state.image_metadata)
            ]
        )
    else:
        image_info = []
        image_text = "현재 업로드된 이미지가 없습니다."

    print(image_text)

    # OpenAI 프롬프트 구성
    message = HumanMessage(
        content=[
            {
                "type": "text",
                "text": f"사용자가 {len(st.session_state.image_metadata)} 개의 이미지를 업로드했습니다.\n{image_text}\n\n사용자의 질문: {prompt}\n",
            },
        ]
        + image_info,  # 이미지 리스트를 추가
    )

    with st.chat_message("assistant"):
        st.markdown("응답 생성 중...")

        # OpenAI API 호출
        result = llm.invoke([message])
        response = result.content.strip()

        # AI 응답 저장 및 표시
        st.markdown(response)
        st.session_state.messages.append({"role": "assistant", "content": response})